In [1]:
import os
import sys
import math
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from datetime import datetime

In [2]:
base_path = os.environ['HOME'] + '/ResNet'
if os.path.isdir(base_path):
  pass
else:
  os.mkdir(base_path)

In [3]:
data = tf.keras.datasets.cifar100.load_data()

train = data[0]
test = data[1]

train_image, train_label = train[0].astype(np.float32), train[1]
test_image, test_label= test[0].astype(np.float32), test[1]

train_label = np.reshape(train_label, newshape=[-1])
test_label = np.reshape(test_label, newshape=[-1])

In [4]:
# Generate Dataset obj   
dataset_obj = tf.data.Dataset.from_tensors(
    {'image': train_image, 'label': train_label})
dataset_obj = dataset_obj.shuffle(50000)
dataset_obj = dataset_obj.unbatch()

# split train-validation dataset
train_dataset = dataset_obj.take(40000)
val_dataset = dataset_obj.skip(40000).take(10000)

test_dataset = tf.data.Dataset.from_tensors(
  {'image': test_image, 'label': test_label})
test_dataset = test_dataset.shuffle(10000).unbatch()

def _preprocessing(dataset, train_mode):
  """
  While train steps, image will be padded random crop and filped(horizontaly)
  And entire steps, per-pixel mean subtracted will be required.
  Args:
    dataset: 'tf.data.Dataset'
    train_mode: 'bool'
  Returns:
    'tf.data.Dataset'
  """
  if train_mode:
    image = dataset['image']
    pad = tf.constant([[2, 2], [2, 2], [0, 0]])
    image = tf.pad(
      tensor=image, paddings=pad)
    image = tf.image.random_crop(
      value=image, size=[32, 32, 3])
    image = tf.image.random_flip_left_right(image=image)
  else:
    image = dataset['image']
    
  label = dataset['label']
  image = tf.math.subtract(
      x=image,
      y=tf.reshape(
          tf.math.reduce_mean(image, axis=2),
          shape=[32, 32, 1]))
  return (image, label)

train_dataset = train_dataset.map(
  lambda x: _preprocessing(x, train_mode=True))
val_dataset = val_dataset.map(
  lambda x: _preprocessing(x, train_mode=False))
test_dataset = test_dataset.map(
  lambda x: _preprocessing(x, train_mode=False))


train_dataset = train_dataset.repeat()
val_dataset = val_dataset.repeat()

In [5]:
# Experiment Parameter
WEIGHT_DECAY_COEFFICIENT = [1e-3, 1e-2, 1e-1, 1]
NUMBER_OF_LAYERS = 50

In [6]:
def residual_block(data, name, weight_decay):
  """
  "bottleneck" building block
  Args:
    data: 'tf.Tensor' generated passing through keras layers
    num_filters: 'int' # of feature map(activation map)
    name: 'str'
  Returns:
    'tf.Tensor' keras layers
  """
  with tf.name_scope(name) as scope:
    identity_data = data
    
    data = keras.layers.Conv2D(
        filters=64, kernel_size=[1, 1], strides=1, padding='same',
        kernel_regularizer=tf.keras.regularizers.l2(weight_decay))(data)
    data = keras.layers.BatchNormalization()(data)
    data = keras.layers.ReLU()(data)
    
    data = keras.layers.Conv2D(
        filters=64, kernel_size=[3, 3], strides=1, padding='same',
        kernel_regularizer=tf.keras.regularizers.l2(weight_decay))(data)
    data = keras.layers.BatchNormalization()(data)
    data = keras.layers.ReLU()(data)
    
    data = keras.layers.Conv2D(
        filters=256, kernel_size=[1, 1], strides=1, padding='same',
        kernel_regularizer=tf.keras.regularizers.l2(weight_decay))(data)
    data = keras.layers.BatchNormalization()(data)
    
    data = keras.layers.Add()([data, identity_data])
    
    data = keras.layers.ReLU()(data)
    return data

In [7]:
wdc = 1e-2

In [8]:
# Ridge Regularization

inputs = keras.Input(shape=[32, 32, 3], name='input_image')

with tf.name_scope("First_block") as scope:
  x = keras.layers.Conv2D(
    filters=256, kernel_size=[2, 2], strides=2,
    kernel_regularizer=tf.keras.regularizers.l2(wdc))(inputs)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.ReLU()(x)
  x = keras.layers.MaxPool2D()(x)


for layers in range(NUMBER_OF_LAYERS):
  x = residual_block(data=x, name="RB{}".format(layers), weight_decay=wdc)

with tf.name_scope("GlobalAveragePooling") as scope:
  x = keras.layers.GlobalAveragePooling2D()(x)
  outputs = keras.layers.Dense(100, activation='softmax', 
      kernel_regularizer=tf.keras.regularizers.l2(wdc))(x)

model = keras.Model(
    inputs, outputs, 
    name="{}layer_{}weight_decay_{}".format(
      NUMBER_OF_LAYERS, wdc,
      datetime.strftime(datetime.now(), "%Y%m%d-%H%M")))

# Callbacks
callbacks_list = [
  keras.callbacks.EarlyStopping(
    monitor='val_sparse_categorical_accuracy', min_delta=1e-2, 
    patience=1000, verbose=1),
  keras.callbacks.ModelCheckpoint(
      filepath=base_path+'/ckpts/{}.h5'.format(model.name),
      verbos=1, save_best_only=True),
  keras.callbacks.TensorBoard(
      log_dir= base_path+'/logs/{}/'.format(model.name),
      histogram_freq=10,
      update_freq='epoch')]

model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=[keras.metrics.SparseCategoricalAccuracy()])


model.fit(
  x=train_dataset.shuffle(1000).batch(100),
  epochs=10000,
  validation_data=val_dataset.shuffle(1000).batch(100),
  validation_freq=1,
  callbacks=callbacks_list,
  steps_per_epoch=100,
  validation_steps=100)

model.evaluate(
x=test_dataset.batch(1000))


Train for 100 steps, validate for 100 steps
Epoch 1/10000
100/100 [==============================] - 62s 620ms/step - loss: 103.1481 - sparse_categorical_accuracy: 0.0438 - val_loss: 67.1385 - val_sparse_categorical_accuracy: 0.0230
Epoch 2/10000
100/100 [==============================] - 36s 360ms/step - loss: 46.9455 - sparse_categorical_accuracy: 0.0772 - val_loss: 32.3727 - val_sparse_categorical_accuracy: 0.0262
Epoch 3/10000
100/100 [==============================] - 36s 359ms/step - loss: 24.1128 - sparse_categorical_accuracy: 0.0915 - val_loss: 18.5303 - val_sparse_categorical_accuracy: 0.0285
Epoch 4/10000
100/100 [==============================] - 36s 363ms/step - loss: 14.6257 - sparse_categorical_accuracy: 0.0992 - val_loss: 12.2971 - val_sparse_categorical_accuracy: 0.0425
Epoch 5/10000
100/100 [==============================] - 36s 363ms/step - loss: 10.1365 - sparse_categorical_accuracy: 0.1042 - val_loss: 9.0913 - val_sparse_categorical_accuracy: 0.0538
Epoch 6/10000
10

100/100 [==============================] - 36s 357ms/step - loss: 3.9367 - sparse_categorical_accuracy: 0.2024 - val_loss: 4.0655 - val_sparse_categorical_accuracy: 0.1683
Epoch 45/10000
100/100 [==============================] - 35s 346ms/step - loss: 3.9391 - sparse_categorical_accuracy: 0.1974 - val_loss: 4.5620 - val_sparse_categorical_accuracy: 0.1220
Epoch 46/10000
100/100 [==============================] - 36s 356ms/step - loss: 3.8838 - sparse_categorical_accuracy: 0.2072 - val_loss: 4.7907 - val_sparse_categorical_accuracy: 0.1252
Epoch 47/10000
100/100 [==============================] - 35s 354ms/step - loss: 3.8954 - sparse_categorical_accuracy: 0.2006 - val_loss: 4.0340 - val_sparse_categorical_accuracy: 0.1805
Epoch 48/10000
100/100 [==============================] - 35s 354ms/step - loss: 3.9364 - sparse_categorical_accuracy: 0.1998 - val_loss: 5.0535 - val_sparse_categorical_accuracy: 0.0937
Epoch 49/10000
100/100 [==============================] - 35s 353ms/step - loss:

Epoch 88/10000
100/100 [==============================] - 35s 355ms/step - loss: 3.4976 - sparse_categorical_accuracy: 0.2413 - val_loss: 3.7020 - val_sparse_categorical_accuracy: 0.1978
Epoch 89/10000
100/100 [==============================] - 35s 353ms/step - loss: 3.5032 - sparse_categorical_accuracy: 0.2320 - val_loss: 3.7013 - val_sparse_categorical_accuracy: 0.2018
Epoch 90/10000
100/100 [==============================] - 35s 353ms/step - loss: 3.4682 - sparse_categorical_accuracy: 0.2497 - val_loss: 4.0811 - val_sparse_categorical_accuracy: 0.1652
Epoch 91/10000
100/100 [==============================] - 36s 358ms/step - loss: 3.4858 - sparse_categorical_accuracy: 0.2386 - val_loss: 3.8681 - val_sparse_categorical_accuracy: 0.1767
Epoch 92/10000
100/100 [==============================] - 35s 351ms/step - loss: 3.4752 - sparse_categorical_accuracy: 0.2429 - val_loss: 4.0833 - val_sparse_categorical_accuracy: 0.1743
Epoch 93/10000
100/100 [==============================] - 35s 351

Epoch 132/10000
100/100 [==============================] - 35s 354ms/step - loss: 3.3539 - sparse_categorical_accuracy: 0.2600 - val_loss: 3.5744 - val_sparse_categorical_accuracy: 0.2202
Epoch 133/10000
100/100 [==============================] - 35s 353ms/step - loss: 3.3414 - sparse_categorical_accuracy: 0.2664 - val_loss: 3.5805 - val_sparse_categorical_accuracy: 0.2190
Epoch 134/10000
100/100 [==============================] - 35s 353ms/step - loss: 3.3441 - sparse_categorical_accuracy: 0.2687 - val_loss: 3.5909 - val_sparse_categorical_accuracy: 0.2316
Epoch 135/10000
100/100 [==============================] - 35s 353ms/step - loss: 3.3467 - sparse_categorical_accuracy: 0.2625 - val_loss: 3.4965 - val_sparse_categorical_accuracy: 0.2372
Epoch 136/10000
100/100 [==============================] - 35s 352ms/step - loss: 3.3446 - sparse_categorical_accuracy: 0.2725 - val_loss: 3.4884 - val_sparse_categorical_accuracy: 0.2367
Epoch 137/10000
100/100 [==============================] - 3

KeyboardInterrupt: 

In [ ]:
"""
estimator = tf.keras.estimator.model_to_estimator(
  keras_model=model,
  model_dir=base_path)

def input_fn():
  data = tf.keras.datasets.cifar100.load_data()
  train = data[0]
  train_image, train_label = train[0].astype(np.float32), train[1]
  train_label = np.reshape(train_label, newshape=[-1])
  
  dataset_obj = tf.data.Dataset.from_tensors(
    {'image': train_image, 'label': train_label})
  dataset_obj = dataset_obj.shuffle(50000)
  dataset_obj = dataset_obj.unbatch()

  train_dataset = dataset_obj.take(40000)
  train_dataset = train_dataset.map(
    lambda x: _preprocessing(x, train_mode=True))
  return train_dataset.repeat().shuffle(1000).batch(100)

early_stopping_hook = tf.estimator.experimental.stop_if_no_decrease_hook(
  estimator=estimator,
  metric_name=['loss'],
  max_steps_without_decrease=10)

ckpt_saver_hook = tf.estimator.CheckpointSaverHook(
  checkpoint_dir=base_path,
  save_steps=1000)

estimator.train(
  input_fn=input_fn,
  hooks=[early_stopping_hook, ckpt_saver_hook])
"""